In [1]:
import json
import cv2
import os
import numpy as np
import torch
import torchvision
import pycocotools
import cv2 as cv
from torchvision import transforms
from torchvision.models.detection.rpn import AnchorGenerator

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Device type:    {device}")

Device type:    cuda


In [ ]:
def get_keypoints_model():
    anchor_generator = AnchorGenerator(sizes=(32, 64, 128, 256, 512), aspect_ratios=(0.25, 0.5, 0.75, 1.0, 2.0, 3.0, 4.0))
    model=torchvision.models.detection.keypointrcnn_resnet50_fpn(pretrained=False,pretrained_backbone=True,num_keypoints=1,num_classes=7,rpn_anchor_generator=anchor_generator)
    return model

In [ ]:
model = get_keypoints_model()

In [ ]:
checkpoint=torch.load("outputs\checkpoint_epoch30.pth")
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

In [3]:
model = torch.load("outputs\kp_model.pth")
model.eval()

KeypointRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(640, 672, 704, 736, 768, 800), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=1e-05)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=1e-05)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=1e-05)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, 

In [4]:
with open("data\img_test_ann.json") as coco_file:
  coco = json.load(coco_file)

In [ ]:
model = model.to(device)

In [5]:
global_path = "data\\test_images"
font                   = cv2.FONT_HERSHEY_SIMPLEX
bottomLeftCornerOfText = (10,30)
fontScale              = 1
fontColor              = (255,255,255)
thickness              = 1
lineType               = 2
categ={1:'oats',2:'tea',3:'crisps',4:'soup',5:'scissors',6:'pan'}
for item in coco["images"]:
  file_name = item["file_name"]
  img_path = os.path.join(global_path,file_name)
  img = cv.imread(img_path)
  imgTens=torchvision.transforms.functional.to_tensor(img)
  imgTens = imgTens.to(device)
  with torch.no_grad():
    output = model([imgTens])
  for idx in range(0,6):
    bbox=output[0]['boxes'][idx]
    cat=output[0]['labels'][idx]
    p0 =output[0]['keypoints'][idx][0]
    # p1 =output[0]['keypoints'][idx][1]
    # p2 =output[0]['keypoints'][idx][2]
    # p3 =output[0]['keypoints'][idx][3]
    # p4 =output[0]['keypoints'][idx][4]
    img= cv2.rectangle(img,(int(bbox[0].item()),int(bbox[1].item())),(int(bbox[2].item()),int(bbox[3].item())),(255,0,255),3)
    img= cv2.circle(img,(int(p0[0]),int(p0[1])),4,(255,255,255),5) #grasping point
    cv2.putText(img,categ[int(cat.item())],(int(bbox[0].item())+40,int(bbox[1].item())) , font, fontScale,fontColor,thickness,lineType)
  cv2.imwrite(os.path.join('tested_net',file_name),img)
                